[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ZGObhOKJCQhJJZFakc-v2ykj-hXm7K2o?usp=sharing)


# Fine-tuning RoBERTa for Commodity Classification with Hugging Face Transformers and ICIS Datasets Library


In [1]:
!pip install -U transformers datasets huggingface_hub tensorboard==2.18.0
!sudo apt-get install git-lfs --yes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [4]:
import os, multiprocessing
import torch
from datasets import load_dataset
from transformers import (
    RobertaForMaskedLM,
    RobertaTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
from huggingface_hub import HfFolder, notebook_login
from google.colab import userdata, drive

In [3]:
notebook_login()

In [5]:
# IDs & hardware params
model_id   = "FacebookAI/roberta-base"
dataset_id    = "Netizine/icis"
repo_id       = "Netizine/icis"     # RE-CREATE THIS REPO EMPTY BEFORE YOU RUN
output_dir    = "output/icis"
# Mount Drive for persistent storage
drive.mount("/content/drive")
# Dynamic CPU counts
num_cpus    = multiprocessing.cpu_count()
num_proc    = max(1, num_cpus - 2)
num_workers = max(1, num_cpus // 2)

Mounted at /content/drive


In [7]:
# Load dataset
train_ds = load_dataset(dataset_id, split="train")

corpus.txt:   0%|          | 0.00/174M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1352658 [00:00<?, ? examples/s]

In [8]:
# Init tokenizer & model
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)
device    = torch.device("cuda")
model     = RobertaForMaskedLM.from_pretrained(model_id).to(device)

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,     # or compute your 90th pct length
        return_special_tokens_mask=True
    )

# Tokenize
tokenized = train_ds.map(
    tokenize_fn,
    batched=True,
    num_proc=num_proc,
    remove_columns=["text"]
).shuffle(seed=42)

# Trainer setup
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.15)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/icis-checkpoints",
    per_device_train_batch_size=32,
    gradient_checkpointing=True,
    fp16=True,
    num_train_epochs=3,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,

    logging_strategy="steps",
    logging_steps=10000,

    save_strategy="steps",
    save_steps=25000,
    save_total_limit=5,

    push_to_hub=True,
    hub_strategy="end",
    hub_model_id=repo_id,
    hub_token=os.getenv("HF_TOKEN"),

    dataloader_num_workers=num_workers,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Map (num_proc=10):   0%|          | 0/1352658 [00:00<?, ? examples/s]

In [10]:
# Fine-tune the model (auto-resume if you re-run with: resume_from_checkpoint=True)
print("▶️ Starting training…")
trainer.train()

# Save our tokenizer and create model card
tokenizer.save_pretrained(repo_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub(commit_message="Pushing final RoBERTa MLM")

# Fine-tune the model
print("✅ Done! The fine-tuned model is saved on Hugging Face Hub.")

▶️ Starting training…


Step,Training Loss
5000,1.856200
10000,1.658300
15000,1.581900
20000,1.524800


Step,Training Loss
5000,1.856200
10000,1.658300
15000,1.581900
20000,1.524800
25000,1.489200
30000,1.451400
35000,1.422000
40000,1.399100
45000,1.377100
50000,1.351200


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

✅ Done! The fine-tuned model is saved on Hugging Face Hub.


In [14]:
# Clear the storage
!ls "/content/drive/MyDrive"


# Replace `icis-checkpoints` with whatever folder you used
!rm -rf "/content/drive/MyDrive/icis-checkpoints"

!ls "/content/drive/MyDrive"


'Bridge Buildings Individuals Survey.gform'   Outlook
'Colab Notebooks'			     'SCM Coding Tests v1009.gdoc'
'Fill With LinkedIn contacts.gsheet'	     'Untitled spreadsheet.gsheet'
'Getting started.pdf'
'Bridge Buildings Individuals Survey.gform'   Outlook
'Colab Notebooks'			     'SCM Coding Tests v1009.gdoc'
'Fill With LinkedIn contacts.gsheet'	     'Untitled spreadsheet.gsheet'
'Getting started.pdf'


In [16]:
# Test our fine-tuned MLM with a fill-mask pipeline

from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=repo_id,
    tokenizer=repo_id,
    device=0  # assumes a single GPU; remove for CPU
)

# Craft a sentence with the special mask token
test_sentence = (
    "The glycerine market in Europe will continue to see [MASK] demand "
    "next year, and imports will provide supply stability."
)

# Run the mask-filling
results = fill_mask(test_sentence)

# Display the top 5 predictions
for res in results:
    print(f"{res['sequence']}  (score: {res['score']:.4f})")

ValueError: Unrecognized model in Netizine/icis. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v3, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, elect